In [ ]:
# tweepy is a python wrapper around Twitter API
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy import API
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re #regular expression

import string
import sys
#import preprocessor as p

import pymongo

In [ ]:
config_file = ".mongodb.json"
with open(config_file) as fh:
    mc = json.load(fh)

client = pymongo.MongoClient(f"{mc['scheme']}://{mc['login']}:{mc['password']}@{mc['url']}")
db = client.nlp_twitter
print(db.list_collection_names())
tweets = db.tweets

In [ ]:
class StreamListener(StreamListener):
    def on_status (self, status):
#         print(status._json)
        tweets.insert_one(status._json)
        print('.')
#         print(status.id_str)
#         print(status.text)

            
    def on_error(self, status_code):
        print("Encountered an error during streaming: ", status_code)
        sys.exit()

In [ ]:
if __name__ == "__main__":
    
    # twitter credentials are saved in the .tweepy.json file as a dictionary
    
    config_file = ".tweepy.json"
    with open(config_file) as fh:
        config = json.load(fh)
    """
    {"access_token"        : '...',
    "access_token_secret"  : '...'
    "consumer_key"       :  '...'
    "consumer_secret"     : '...'}
    """
    # pass twitter credentials to tweepy, complete authorization and initialize API endpoint
    
    auth = OAuthHandler(config['consumer_key'], config['consumer_secret'])
    auth.set_access_token(config['access_token'], config['access_token_secret'])
    """
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    """
    api = API(auth)
    
    # using update_status of api, you can tweet to your twitter account from your code
    # tweet = api.update_status("Tweeting from the code again in the middle of the night!")
    # initialize stream
    streamListener = StreamListener()
    stream = Stream(auth = api.auth, listener = streamListener, tweed_mode = 'extended', verify=False)
    
    tracklist = ['corona', 'covid19']
    stream.filter(track = tracklist)